# PC2 - Grupo 2 (Python)

#### Integrantes
- GARCIA RODRIGUEZ, EMILIO ALONSO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SALAMANCA FERNANDEZ, LUCAS PABLO
- SILVA ANDUJAR, NICOLAS

#### 1. Loading and processing the data


In [3]:
import numpy as np
import pandas as pd

In [5]:
data= "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"

df =pd.read_csv(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5150 entries, 0 to 5149
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rownames  5150 non-null   int64  
 1   wage      5150 non-null   float64
 2   lwage     5150 non-null   float64
 3   sex       5150 non-null   float64
 4   shs       5150 non-null   float64
 5   hsg       5150 non-null   float64
 6   scl       5150 non-null   float64
 7   clg       5150 non-null   float64
 8   ad        5150 non-null   float64
 9   mw        5150 non-null   float64
 10  so        5150 non-null   float64
 11  we        5150 non-null   float64
 12  ne        5150 non-null   float64
 13  exp1      5150 non-null   float64
 14  exp2      5150 non-null   float64
 15  exp3      5150 non-null   float64
 16  exp4      5150 non-null   float64
 17  occ       5150 non-null   float64
 18  occ2      5150 non-null   int64  
 19  ind       5150 non-null   float64
 20  ind2      5150 non-null   int6

In [6]:
df = df.set_index('rownames')

#### As in Group Assignment 1 2024 - 2 #1044 , generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

In [12]:
df_with_dummies = pd.get_dummies(df, columns=['occ2', 'ind2'], drop_first=True)
extra_flexible_model_vars = ['exp1', 'exp2', 'exp3', 'exp4', 'hsg', 'scl', 'clg', 'ad', 
                             'so', 'we', 'ne'] + \
                             [col for col in df_with_dummies.columns if col.startswith('occ2_') or col.startswith('ind2_')]

two_way_interactions = []
for i, var1 in enumerate(extra_flexible_model_vars):
    for var2 in extra_flexible_model_vars[i+1:]:
        interaction_var = df_with_dummies[var1] * df_with_dummies[var2]
        two_way_interactions.append(interaction_var.values.reshape(-1, 1))

interactions_array = np.hstack(two_way_interactions)

extra_flexible_model_array = np.hstack([df_with_dummies[extra_flexible_model_vars].values, interactions_array])
print(extra_flexible_model_array.shape)

(5150, 1378)


#### 2.1. Generate the array for the outcome variable Y and normalize it

In [18]:
df_logwage = df[['lwage']]
log_w = np.array(df_logwage['lwage'])
log_w = log_w.reshape(-1, 1)

In [31]:
def normalize_array(arr):
    min_vals = np.min(arr, axis=0)
    max_vals = np.max(arr, axis=0)
    # Calcular el rango y evitar la división por cero añadiendo un pequeño valor epsilon
    range_vals = max_vals - min_vals
    range_vals[range_vals == 0] = 1  # Esto previene la división por cero
    norm_arr = (arr - min_vals) / range_vals
    return norm_arr

In [32]:
norm_log_w =normalize_array(log_w)
print(norm_log_w)

[[0.22410474]
 [0.53572233]
 [0.25116529]
 ...
 [0.49251752]
 [0.46267104]
 [0.33791134]]


In [33]:
x_data =normalize_array(extra_flexible_model_array)
print(x_data)

[[0.14893617021276595 0.022181982797645994 0.0033036995656068506 ... 0.0
  0.0 0.0]
 [0.6595744680851063 0.435038478949751 0.2869402733498358 ... 0.0 0.0 0.0]
 [0.3829787234042553 0.14667270258035311 0.05617252439247566 ... 0.0 0.0
  0.0]
 ...
 [0.23404255319148937 0.05477591670439112 0.012819895398900051 ... 0.0
  0.0 0.0]
 [0.2127659574468085 0.04526935264825713 0.009631777159203646 ... 0.0 0.0
  0.0]
 [0.2978723404255319 0.08872793119058398 0.026429596524854805 ... 0.0 0.0
  0.0]]


#### Split between training and testing samples. The testing sample should be 10% of the total.

In [35]:
from sklearn.model_selection import train_test_split
ef_model =x_data
y=norm_log_w

In [36]:
ef_train,ef_test,y_ef_train,y_ef_test = train_test_split(ef_model,y, train_size= 0.1)
